In [8]:
from numpy import exp, zeros, ones, mean, ones_like,square,linspace,sqrt,zeros,array
from numpy.random import randint,uniform,normal
from numpy.linalg import norm
from tabulate import tabulate
import time

In [2]:
def obtain_all_params() -> dict:
    """
    Funcion que reune los parametros de la función, su gradiente y de los métodos a utilizar
    """
    params = {
        "max_iter": 100,
        "n": 100,
        "m": 2000,
        "sigma": 1,
        "epsilon": 0.01,
        "theta": 10 * normal(size=2)
    }

    # parámetros del algoritmo
    gd_params = {
        'alpha': 0.95,
        'alphaADADELTA': 0.7,
        'alphaADAM': 0.95,
        'nIter': 300,
        'batch_size': 100,
        'eta': 0.9,
        'eta1': 0.9,
        'eta2': 0.999
    }
    return params, gd_params

In [3]:
class function_class:
    def __init__(self) -> None:
        pass

    def update_phi(self, Y, mu, sigma, n):
        '''
        Construye  Matriz de Kerneles Phi

        Parámetros
        -----------
        + Y: Patrones a Aproximar
        + mu: Array de medias
        + sigma: Vector de Desviaciones
        + n: Número de funciones radiales usadas
        Regresa
        -----------
            phi          : matriz de kerneles
        '''
        phi = zeros((Y.shape[0], n))
        for i in range(n):
            phi[:, i] = exp(- 1.0 / (2*sigma**2) * (Y - mu[i])**2)
        return phi

    def grad_gaussian_radial_mu(self, theta, f_params):
        '''
        Calcula el gradiente respecto a mu
        Parámetros
        -----------
        + theta
        + f_params: lista de parametros para la funcion objetivo,
        + kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
        + X = f_params['X'] Variable independiente
        + y = f_params['y'] Variable dependiente

        Regresa
        -----------
            Array gradiente
        '''
        # Obtengo Parámetros
        phi = f_params['X']
        alpha = f_params['Alpha']
        n = f_params['n']
        Y = f_params['y']
        mu = f_params['mu']
        alpha = alpha.reshape((-1, 1))
        mu = mu.reshape((-1, 1))
        Y = Y.reshape((-1, 1))
        gradient = (phi @ alpha - Y) @ alpha.T * \
            (Y @ ones((1, n)) - ones_like(Y) @ mu.T)
        return mean(gradient, axis=0)

    def grad_gaussian_radial_alpha(self, theta, f_params):
        '''
        Calcula el gradiente respecto a alpha
        Parámetros
        -----------
        + theta
        + f_params: lista de parametros para la funcion objetivo,
        + kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
        + X = f_params['X'] Variable independiente
        + y = f_params['y'] Variable dependiente

        Regresa
        -----------
            Array gradiente
        '''
        # Obtengo Parámetros
        phi = f_params['X']
        Y = f_params['y']
        alpha = f_params['Alpha']
        gradient = phi.T @ (phi @ alpha - Y)
        return mean(gradient, axis=0)

In [4]:
class model_class:
    def __init__(self) -> None:
        """
        Modelo que reune los metodos de 
        + Descenso de gradiente estocástico.
        + Descenso de gradiente estoc ́astico accelerado de tipo Nesterov.
        + AdaDelta
        + ADAM
        """
        pass

    def select_method(self, method_name: str):
        if method_name == "SGD":
            self.method = self.SGD
        if method_name == "NAG":
            self.method = self.NAG
        if method_name == "ADADELTA":
            self.method = self.ADADELTA
        if method_name == "ADAM":
            self.method = self.ADAM

    def SGD(self, theta: list, grad, gd_params: dict, f_params: dict,) -> array:
        """
        Descenso de gradiente estocástico

        Parámetros
        -----------
        theta     :   condicion inicial
        grad      :   funcion que calcula el gradiente

        gd_params :   lista de parametros para el algoritmo de descenso,
                        nIter = gd_params['nIter'] número de iteraciones
                        alpha = gd_params['alpha'] tamaño de paso alpha
                        batch_size = gd_params['batch_size'] tamaño de la muestra

        f_params  :   lista de parametros para la funcion objetivo,
                        kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
                        X     = f_params['X'] Variable independiente
                        y     = f_params['y'] Variable dependiente

        Regresa
        -----------
        Theta     :   trayectoria de los parametros
                        Theta[-1] es el valor alcanzado en la ultima iteracion
        """
        (high, dim) = f_params['X'].shape
        batch_size = gd_params['batch_size']
        nIter = gd_params['nIter']
        alpha = gd_params['alpha']
        Theta = []
        for t in range(nIter):
            # Set of sampled indices
            smpIdx = randint(low=0,
                             high=high,
                             size=batch_size,
                             dtype='int32')
            # sample
            smpX = f_params['X'][smpIdx]
            smpy = f_params['y'][smpIdx]
            # parametros de la funcion objetivo
            smpf_params = {'kappa': f_params['kappa'],
                           "Alpha": f_params["Alpha"],
                           "mu": f_params["mu"],
                           "n": f_params["n"],
                           'X': smpX,
                           'y': smpy}
            p = grad(theta,
                     f_params=smpf_params)
            theta = theta - alpha*p
            Theta.append(theta)
        return array(Theta)

    def NAG(self, theta: list, grad, gd_params: dict, f_params: dict,):
        """
        Descenso acelerado de Nesterov

        Parámetros
        -----------
        theta     :   condicion inicial
        grad      :   funcion que calcula el gradiente
        gd_params :   lista de parametros para el algoritmo de descenso,
                        nIter = gd_params['nIter'] número de iteraciones
                        alpha = gd_params['alpha'] tamaño de paso alpha
                        eta   = gd_params['eta']  parametro de inercia (0,1]
        f_params  :   lista de parametros para la funcion objetivo,
                        kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
                        X     = f_params['X'] Variable independiente
                        y     = f_params['y'] Variable dependiente

        Regresa
        -----------
        Theta     :   trayectoria de los parametros
                        Theta[-1] es el valor alcanzado en la ultima iteracion
        """
        nIter = gd_params['nIter']
        alpha = gd_params['alpha']
        eta = gd_params['eta']
        p = zeros(theta.shape)
        Theta = []
        for t in range(nIter):
            pre_theta = theta - 2.0*alpha*p
            g = grad(pre_theta,
                     f_params=f_params)
            p = g + eta*p
            theta = theta - alpha*p
            Theta.append(theta)
        return array(Theta)

    def ADADELTA(self, theta: list, grad, gd_params: dict, f_params: dict,):
        """
        Descenso de Gradiente Adaptable (ADADELTA)

        Parámetros
        -----------
        theta     :   condicion inicial
        grad      :   funcion que calcula el gradiente
        gd_params :   lista de parametros para el algoritmo de descenso,
                        nIter    = gd_params['nIter'] número de iteraciones
                        alphaADA = gd_params['alphaADADELTA'] tamaño de paso alpha
                        eta      = gd_params['eta']  parametro adaptación del alpha
        f_params  :   lista de parametros para la funcion objetivo,
                        kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
                        X     = f_params['X'] Variable independiente
                        y     = f_params['y'] Variable dependiente

        Regresa
        -----------
        Theta     :   trayectoria de los parametros
                        Theta[-1] es el valor alcanzado en la ultima iteracion
        """
        epsilon = 1e-8
        nIter = gd_params['nIter']
        alpha = gd_params['alphaADADELTA']
        eta = gd_params['eta']
        G = zeros(theta.shape)
        g = zeros(theta.shape)
        Theta = []
        for t in range(nIter):
            g = grad(theta,
                     f_params=f_params)
            G = eta*g**2 + (1-eta)*G
            p = 1.0/(sqrt(G)+epsilon)*g
            theta = theta - alpha * p
            Theta.append(theta)
        return array(Theta)

    def ADAM(self, theta: list, grad, gd_params: dict, f_params: dict,):
        """
        Descenso de Gradiente Adaptable con Momentum(A DAM)

        Parámetros
        -----------
        theta     :   condicion inicial
        grad      :   funcion que calcula el gradiente
        gd_params :   lista de parametros para el algoritmo de descenso,
                        nIter    = gd_params['nIter'] número de iteraciones
                        alphaADA = gd_params['alphaADAM'] tamaño de paso alpha
                        eta1     = gd_params['eta1'] factor de momentum para la direccion
                                    de descenso (0,1)
                        eta2     = gd_params['eta2'] factor de momentum para la el
                                    tamaño de paso (0,1)
        f_params  :   lista de parametros para la funcion objetivo,
                        kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
                        X     = f_params['X'] Variable independiente
                        y     = f_params['y'] Variable dependiente

        Regresa
        -----------
        Theta     :   trayectoria de los parametros
                        Theta[-1] es el valor alcanzado en la ultima iteracion
        """
        epsilon = 1e-8
        nIter = gd_params['nIter']
        alpha = gd_params['alphaADAM']
        eta1 = gd_params['eta1']
        eta2 = gd_params['eta2']
        p = zeros(theta.shape)
        v = 0.0
        Theta = []
        eta1_t = eta1
        eta2_t = eta2
        for t in range(nIter):
            g = grad(theta,
                     f_params=f_params)
            p = eta1*p + (1.0-eta1)*g
            v = eta2*v + (1.0-eta2)*(g**2)
            theta = theta - alpha * p / (sqrt(v)+epsilon)
            eta1_t *= eta1
            eta2_t *= eta2
            Theta.append(theta)
        return array(Theta)

In [5]:
def solver(models: model_class, Y: list, params: dict, gd_params: dict):
    """"
    Modelo que resuelve el problema dado con una serie de metodos

    Parámetros
    --------------------------
    + models: clase que reune a todos los metodos para obtener la solucion del problema
    + Y: datos a aproximar
    + params: parametros del modelo a utilizar
    + gd_params: parametros del metodo a utilizar

    Regresa
    ---------------------------
        phi, alpha y tiempo de ejeccion del metodo
    """"
    max_iter = params["max_iter"]
    epsilon = params["epsilon"]
    sigma = params["sigma"]
    n = params["n"]
    functions = function_class()
    t_init = time.clock_gettime(0)
    # Valores Iniciales
    mu = linspace(0, 100, n)
    phi = functions.update_phi(Y, mu, sigma, n)
    alpha = uniform(0, sigma, n)
    # Parámetros para el gradiente
    f_params = {
        'kappa': 0.01,
        'mu': mu,
        'X': phi,
        'y': Y,
        'Alpha': alpha,
        'n': n
    }
    num_iter = 0
    while num_iter < max_iter:
        # descenso para alpha
        alpha = models.method(alpha,
                              grad=functions.grad_gaussian_radial_alpha,
                              gd_params=gd_params,
                              f_params=f_params)[-1]
        if norm(phi @ alpha - Y) < epsilon:
            break
        # DESCENSO  PARA  MU
        mu_old = mu
        mu = models.method(mu,
                           grad=functions.grad_gaussian_radial_mu,
                           gd_params=gd_params,
                           f_params=f_params)[-1]
        # actualizacion
        phi = functions.update_phi(Y, mu, sigma, n)
        # Criterio de parada
        if norm(mu - mu_old) < epsilon:
            break
        # Número máximo de iteraciones si no hay convergencia
        num_iter += 1
    t_end = time.clock_gettime(0)
    total_time = t_end - t_init
    return phi, alpha, total_time

In [9]:
model_names = ["SGD",
               "NAG",
               "ADAM",
               "ADADELTA"]
# inicializacion del guardado de los resultados
results = {}
# Lecutura de los parametros
params, gd_params = obtain_all_params()
# Inicializacion de los metodos
models = model_class()
y = uniform(0, 1, params["m"])
# recorrido por todos los modelos
for model_name in model_names:
    print("Resolviendo por medio de {}".format(model_name))
    # Inicializacion del guardado de cada metodo
    results[model_name] = {}
    # Seleccion del metodo
    models.select_method(model_name)
    # Solver del problema
    phi, alpha, total_time = solver(models, y, params, gd_params)
    # Error
    error = round(square(phi @ alpha - y).mean(), 8)
    # Guardado de los resultados
    results[model_name]["time"] = total_time
    results[model_name]["error"] = error
# Organizacion para el guardado de los resultados
table = []
for model_name in model_names:
    total_time = results[model_name]["time"]
    error = results[model_name]["error"]
    table += [[model_name, total_time, error]]
print(tabulate(table,
               headers=["Model",
                        "Time",
                        "Error"]))

Resolviendo por medio de SGD
Resolviendo por medio de NAG
Resolviendo por medio de ADAM
Resolviendo por medio de ADADELTA
Model         Time     Error
--------  --------  --------
SGD        16.9738  0.342642
NAG       128.874   0.342642
ADAM      129.253   0.342642
ADADELTA  126.732   0.342642
